In [101]:
import json
from collections import Counter

In [102]:
with open('../data/hashtags/stitch/vertices/sources/maga.json') as f:
    sources = json.load(f)

with open('../data/hashtags/stitch/vertices/targets/maga.json') as f:
    targets = json.load(f)

with open('../data/hashtags/stitch/edges/maga_edges.txt') as f:
    edges = f.readlines()

In [103]:
stitcher, stitchee = zip(*[edge.strip().split(',') for edge in edges])
stitcher = [s.split('/')[-1] for s in stitcher]
stitchee = [s.split('/')[-1] for s in stitchee]

# convert stitchers and stitchees to integers if not 'None'
stitcher = [int(s) if s != 'None' else None for s in stitcher]
stitchee = [int(s) if s != 'None' else None for s in stitchee]

In [104]:
data = {}

for video in sources + targets:
    data[video['id']] = video

    if video['id'] in stitcher:
        data[video['id']]['is_stitcher'] = True
        for i in range(len(stitcher)):
            if stitcher[i] == video['id']:
                data[video['id']]['stitches'] = stitchee[i]
    else:
        data[video['id']]['is_stitcher'] = False
    
    if video['id'] in stitchee:
        data[video['id']]['is_stitchee'] = True
        for i in range(len(stitchee)):
            if stitchee[i] == video['id']:
                if 'stitchers' in data[video['id']]:
                    data[video['id']]['stitchers'].append(stitcher[i])
                else:
                    data[video['id']]['stitchers'] = [stitcher[i]]
    else:
        data[video['id']]['is_stitchee'] = False

In [109]:
list(data.values())[2]

{'username': 'biggenhaulen',
 'video_description': '#stitch with @MAGA 👑 KING 👑 🇺🇸#fypシ゚viral #maga #Trump #trump2024 #draintheswamp ',
 'view_count': 329,
 'create_time': 1717195830,
 'hashtag_names': ['stitch',
  'trump',
  'maga',
  'draintheswamp',
  'trump2024',
  'fypシ゚viral'],
 'id': 7375299912499612974,
 'is_stem_verified': False,
 'region_code': 'US',
 'is_stitcher': True,
 'stitches': 7370132597608828206,
 'is_stitchee': False}

In [113]:
data[7370338195956944170]

{'is_stem_verified': False,
 'region_code': 'us',
 'username': 'drewcifers',
 'video_description': '#daddydrewcifers ',
 'view_count': 559,
 'create_time': 1716040604,
 'hashtag_names': ['daddydrewcifers'],
 'id': 7370338195956944170,
 'is_stitcher': False,
 'is_stitchee': True,
 'stitchers': [7370344417837813038]}